# 01 - Bigram Language Model
Following along with Karpathy's "Let's build GPT from scratch" video.

Video: https://www.youtube.com/watch?v=kCc8FmEb1nY

In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Check device
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f'Using device: {device}')

# Read the dataset
with open('../data/shakespeare/input.txt', 'r') as f:
    text = f.read()

print(f'Length of dataset: {len(text):,} characters')
print(f'First 200 chars:\n{text[:200]}')

Using device: cpu
Length of dataset: 1,115,394 characters
First 200 chars:
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [4]:
# Build character vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f'Vocab size: {vocab_size}')
print(''.join(chars))

# Create encode/decode mappings
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# Train/val split
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

Vocab size: 65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [5]:
# Hyperparameters
batch_size = 32
block_size = 8
learning_rate = 1e-3
max_iters = 3000
eval_interval = 300
eval_iters = 200

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [7]:
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [8]:
block_size = 8
train_data[0:9]
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
torch.manual_seed(1000)
batch_size = 4 # indepeendent sequence to process in parallel
block_size = 8 # max context lenght

data = train_data #if split == 'train' else val_data

ix = torch.randint(len(data) - block_size, (batch_size,))
xb = torch.stack([data[i:i+block_size] for i in ix])
yb = torch.stack([data[i+1:i+block_size+1] for i in ix])
print(len(data),ix)
print(xb)
print(yb)

1003854 tensor([392491, 328445,  58815, 532942])
tensor([[47, 57, 46,  1, 40, 50, 53, 53],
        [56, 39, 63,  5, 57,  1, 40, 56],
        [ 1, 15, 47, 58, 47, 64, 43, 52],
        [54, 43,  6,  1, 58, 46, 53, 59]])
tensor([[57, 46,  1, 40, 50, 53, 53, 42],
        [39, 63,  5, 57,  1, 40, 56, 43],
        [15, 47, 58, 47, 64, 43, 52, 10],
        [43,  6,  1, 58, 46, 53, 59,  1]])


In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        #each token directly reads of logits of neext token
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are (B,T) tensors of integers
        logits = self.token_embedding_table(idx) #(B,T,C)

        if targets is None:
            loss = None

        else:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))

        return logits, loss

    def generate(self, idx, max_new_tokens):

        #idx is (B,T) array of indices
        for _ in range(max_new_tokens):

            #get predictions 
            logits, loss = self(idx)

            #focusing on last time step only
            logits = logits[:, -1, :] #becomes (B,C)

            #apply softmax to get probability
            probs = F.softmax(logits, dim=-1) #(B.C)

            #sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)

            #append sample index to running sequence
            idx = torch.cat((idx,idx_next), dim=1) #(B,T+1)
        return idx
        

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
print(loss)
print(logits.shape)
# print(logits)


idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx,max_new_tokens=100)[0].tolist()))

tensor(4.6665, grad_fn=<NllLossBackward0>)
torch.Size([4, 8, 65])

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [11]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [12]:
# Training loop (uncomment after building the model)
# model = BigramLanguageModel(vocab_size).to(device)
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
#
for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss(m)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.7344, val loss 4.7194
step 300: train loss 4.5317, val loss 4.5103
step 600: train loss 4.2892, val loss 4.2967
step 900: train loss 4.1011, val loss 4.1359
step 1200: train loss 3.9181, val loss 3.9245
step 1500: train loss 3.7812, val loss 3.7977
step 1800: train loss 3.6359, val loss 3.6258
step 2100: train loss 3.5065, val loss 3.5221
step 2400: train loss 3.4113, val loss 3.4108
step 2700: train loss 3.2904, val loss 3.3001


In [13]:
# Generate text (uncomment after training)
# context = torch.zeros((1, 1), dtype=torch.long, device=device)
# print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

In [14]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx,max_new_tokens=400)[0].tolist()))


Cj?pVGLRWTKuATM.t;w.SKprw wNCW?acQXW.?ghs,,StUboivz;iFd'dHAr -v$NHAIsyxER:CEH.?athaPo'sWFbola!AtbVz,mZhinnotliP3T:ubqA!rdf!zP:kqqfq-XLSBo.kFlviGooTERjXcmszLq?Ja!-IUkHAO:VjxIfUgmer ATbf swQF-taUvUTqzLve'apw
pCGwrJ$HEDir mWXcu slLkRGSX$
S!atD& nmen;Pa
xresperbas,SqqmbSJMHenaPcevCnglLLK't IGSSAc:O3vi

EJJcmpXimwOpmye:C?ul:C'tsatX&y KZetoX3fNeUix'QUAng sERUd. i-WtST:witrsVQXGZ;'Vjs, faYXR;f,NBZApa
CPa


In [21]:
tril = torch.tril(torch.ones(5,5))

In [22]:
tril

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [28]:
wei = torch.zeros((5,5)) # token wieghts, how much weigh to give eeach token
wei

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [29]:
wei = wei.masked_fill(tril==0, float('-inf')) # tokens from past can't communicate
wei

tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]])

In [31]:
wei = F.softmax(wei, dim=-1) # it takes exponential and then avergaes it out
wei

tensor([[0.4046, 0.1488, 0.1488, 0.1488, 0.1488],
        [0.2618, 0.2618, 0.1588, 0.1588, 0.1588],
        [0.2256, 0.2256, 0.2256, 0.1616, 0.1616],
        [0.2093, 0.2093, 0.2093, 0.2093, 0.1630],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000]])